In [21]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [22]:
import datetime
import pandas as pd

In [23]:
import re

date_pattern = re.compile('([0-9]+)/([0-9]+)/([0-9]+)')
time_pattern = re.compile('([0-9]+):([0-9]+)')
mothermilk_pattern_right = re.compile('.*右 ([0-9]+)分')
mothermilk_pattern_left = re.compile('.*左 ([0-9]+)分')
milk_pattern = re.compile('.*ミルク ([0-9]+)ml')
expressedmilk_pattern = re.compile('.*搾母乳 ([0-9]+)ml')
small_pattern = re.compile('.*おしっこ')
big_pattern = re.compile('.*うんち')
sleep_pattern_in = re.compile('.*寝る')
sleep_pattern_out = re.compile('.*起きる')

In [24]:
target = '202103'

In [25]:
df = pd.DataFrame(columns=['date', 'time', 'mothermilk', 'milk', 'expressedmilk', 'small', 'big', 'sleep'])

with open(f'data/raw/{target}.txt', 'r', encoding='UTF-8') as f:
    lines = f.readlines()
    for l in lines:
        date_result = date_pattern.match(l)
        if date_result:
            date = datetime.date(int(date_result.group(1)), int(date_result.group(2)), int(date_result.group(3)))
            print(date)

        time_result = time_pattern.match(l)
        if time_result:
            time = datetime.time(int(time_result.group(1)), int(time_result.group(2)))
        
        mothermilk_min = 0
        mothermilk_result_right = mothermilk_pattern_right.match(l)
        if mothermilk_result_right:
            mothermilk_min = mothermilk_min + int(mothermilk_result_right.group(1))
        mothermilk_result_left = mothermilk_pattern_left.match(l)
        if mothermilk_result_left:
            mothermilk_min = mothermilk_min + int(mothermilk_result_left.group(1))
        if mothermilk_min > 0:
            df = df.append({
                'date': date,
                'time': time,
                'mothermilk': mothermilk_min
            }, ignore_index=True)
        
        milk_result = milk_pattern.match(l)
        if milk_result:
            df = df.append({
                'date': date,
                'time': time,
                'milk': int(milk_result.group(1))
            }, ignore_index=True)
        
        expressedmilk_result = expressedmilk_pattern.match(l)
        if expressedmilk_result:
            df = df.append({
                'date': date,
                'time': time,
                'expressedmilk': int(expressedmilk_result.group(1))
            }, ignore_index=True)
        
        small_result = small_pattern.match(l)
        if small_result:
            df = df.append({
                'date': date,
                'time': time,
                'small': 1
            }, ignore_index=True)
        
        big_result = big_pattern.match(l)
        if big_result:
            df = df.append({
                'date': date,
                'time': time,
                'big': 1
            }, ignore_index=True)
        
        sleep_result_in = sleep_pattern_in.match(l)
        if sleep_result_in:
            df = df.append({
                'date': date,
                'time': time,
                'sleep': 1
            }, ignore_index=True)
        sleep_result_out = sleep_pattern_out.match(l)
        if sleep_result_out:
            df = df.append({
                'date': date,
                'time': time,
                'sleep': 0
            }, ignore_index=True)

df

2021-03-01
2021-03-02
2021-03-03
2021-03-04
2021-03-05
2021-03-06
2021-03-07
2021-03-08
2021-03-09
2021-03-10
2021-03-11
2021-03-12
2021-03-13
2021-03-14
2021-03-15
2021-03-16
2021-03-17
2021-03-18
2021-03-19
2021-03-20
2021-03-21
2021-03-22
2021-03-23
2021-03-24
2021-03-25
2021-03-26
2021-03-27
2021-03-28
2021-03-29
2021-03-30
2021-03-31


,date,time,mothermilk,milk,expressedmilk,small,big,sleep
0,2021-03-01,05:15:00,NaN,NaN,NaN,NaN,NaN,0
1,2021-03-01,05:30:00,NaN,NaN,NaN,1,NaN,NaN
2,2021-03-01,05:55:00,6,NaN,NaN,NaN,NaN,NaN
3,2021-03-01,06:25:00,NaN,NaN,NaN,1,NaN,NaN
4,2021-03-01,08:00:00,NaN,NaN,NaN,1,NaN,NaN
...,...,...,...,...,...,...,...,...
772,2021-03-31,21:45:00,NaN,NaN,NaN,NaN,NaN,0
773,2021-03-31,21:50:00,NaN,NaN,NaN,NaN,NaN,1
774,2021-03-31,21:50:00,9,NaN,NaN,NaN,NaN,NaN
775,2021-03-31,21:50:00,NaN,NaN,NaN,1,NaN,NaN


In [26]:
df.to_csv(f'data/parsed/{target}.tsv', sep='\t')